In [1]:
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd
import datetime
import sys
import re
import gc
import glob

import os
HOME = os.path.expanduser('~')
sys.path.append(f"{HOME}/kaggle/data_analysis/library/")
import utils
from utils import logger_func
logger = logger_func()
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
from sklearn.metrics import mean_squared_error


path_list = glob.glob('../stack/*.gz')
import pickle
import datetime
from tqdm import tqdm
start_time = "{0:%Y%m%d_%H%M%S}".format(datetime.datetime.now())
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_active_month', 'index', 'personal_term']

2019-02-26 13:23:36,739 utils 339 [INFO]    [logger_func] start 


In [11]:
#========================================================================
# CV CHECKER 
# Data Load
def get_cv_score(model_path):
    base = utils.read_df_pkl('../input/base_term*')[[key, target]].set_index(key)
    train = base[~base[target].isnull()]

    tmp = utils.read_pkl_gzip(model_path)
    if 'pred_mean' in tmp.columns:
        pred_col = 'pred_mean'
    elif 'prediction' in tmp.columns:
        pred_col = 'prediction'
    tmp = tmp.reset_index()[[key, pred_col]].set_index(key)
    
    train['pred'] = tmp[pred_col]

    y_train = train[target].values
    y_pred = train['pred'].values
    score = np.sqrt(mean_squared_error(y_train, y_pred))
    print(f"CV: {score}")
    return score
#========================================================================


#========================================================================
# 相関

def get_corr_ensemble(path, base, out_part = ''):
    '''
    1. ベースモデルとの相関を個別に取得する
    2. ベースモデルにpath_listのモデルの予測をjoinし、相関行列を取得する
    '''
    
    if key in base.columns:
        base = base.set_index(key)
    
    #========================================================================
    # 個別に相関を見ていく場合
    if str(type(path)).count('str'):
    
        if path.count('CV1'):
            return 0, ''
        if path.count('binary'):
            return 0, ''
        tmp = utils.read_pkl_gzip(path)
    
        try:
            tmp = tmp.reset_index()[[key, 'pred_mean']].set_index(key)
            base['tmp_pred'] = tmp.reset_index()['pred_mean']
        except KeyError:
            tmp = tmp.reset_index()[[key, 'prediction']].set_index(key)
            base['tmp_pred'] = tmp['prediction']
    
        if path.count('LB'):
            try:
                cv2 = re.search(r'CV([^/.]*)_LB.gz', path.replace('.', '-')).group(1)
            except AttributeError:
                return 0, ''
        else:
            cv2 = re.search(r'CV([^/.]*)', path.replace('.', '-')).group(1)
    
        corr = np.corrcoef(base['base_pred'], base['tmp_pred'].values).min()
    
        if corr>0 and corr<0.98:
            logger.info(f"CORR: {corr} | CV{cv1[:6]} vs CV{cv2[:6]}")
            
            return corr, path
        else:
            return 0, ''
    #========================================================================
    
    #========================================================================
    # まとめて相関行列を見たい場合
    elif str(type(path)).count('list'):
        path_list = path
        for path in path_list:
        
            if out_part=='all':
                if path.count('CV-1') or path.count('binary'):
                    continue
            elif out_part=='rm_out':
                if path.count('CV-3') or path.count('binary'):
                    continue
            elif out_part=='binary':
                if path.count('CV-3') or path.count('CV-1'):
                    continue
            tmp = utils.read_pkl_gzip(path)

            if 'pred_mean' in tmp.columns:
                pred_col = 'pred_mean'
            elif 'prediction' in tmp.columns:
                pred_col = 'prediction'
                
            tmp = tmp.reset_index()[[key, pred_col]].set_index(key)
                
            if path.count('LB'):
                try:
                    cv2 = re.search(r'CV([^/.]*)_LB', path.replace('.', '-')).group(1)
                except AttributeError:
                    print(path.replace('.', '-'))
                    continue
            else:
                cv2 = re.search(r'CV([^/.]*)', path.replace('.', '-')).group(1)
                
            base[f'pr_{cv2[:9]}'] = tmp[pred_col]
            
        if out_part=='rm_out':
            base = base[base[target]>-30]
        else:
            base = base[base[target]>-30]
            pass
            
        drop_cols = [col for col in base.columns if col.count('term') or col.count('no_out_flg') or col in ignore_list or col.count('clf')]
        base.drop(drop_cols, axis=1, inplace=True)
        base.sort_index(axis=1, inplace=True)
        mx_corr = base.corr(method='pearson')
#         mx_cor = mx_corr.drop(drop_cols, axis=1).loc[~mx_corr.index.isin(drop_cols)]
        
        return mx_corr
#========================================================================

In [12]:
is_clf = 0
base = utils.read_df_pkl('../input/base_no_out*')[[key, target, 'clf_pred', 'no_out_flg']].set_index(key)
# LB3664 = utils.read_pkl_gzip('../ensemble/good_submit_ensemble/0215_083_stack_submit_OUT29-7593_CV3-61266_LB3-664.gz').set_index(key)
# base['lb3664'] = LB3664['prediction']
#========================================================================
# rm_out以外の部分の相関を見る
# base = base[base['clf_pred']>=0.01]
#========================================================================
# ens_list = glob.glob('../stack/*lgb*all*CV3*.gz')
# ens_list = glob.glob('../model/LB3664_set/*_lgb_*.gz')
# ens_list = glob.glob('../ensemble/lgb_ensemble/*_lgb_*.gz')
ens_list = glob.glob('../ensemble/dir_stack_blend/*_lgb_*.gz')
# ens_list = glob.glob('../ensemble/rm_outlier_ensemble/*_lgb_*.gz')
out_part = 'all'
# ens_list += glob.glob('../stack/0213_2*no_out*.gz') + glob.glob('../stack/0214_*no_out*.gz')

if is_clf:
    out_part = 'binary'
#     ens_list = glob.glob('../stack/0214*NN*-clf*.gz')
    ens_list = glob.glob('../ensemble/binary_ensemble/*.gz')
    base[target] = base[target].map(lambda x: 1 if x<-30 else 0)
top_corr = get_corr_ensemble(path=ens_list, base=base, out_part=out_part)
print(top_corr.shape)
display(top_corr.head())
# top_corr.to_csv('../output/0214_07_elo_rm_out_corr_ensemble_heatmap.csv', index=True)

100%|██████████| 1/1 [00:00<00:00,  4.36it/s]


(80, 80)


,pr_3-6154197,pr_3-6163984,pr_3-6175988,pr_3-6181787,pr_3-6195263,pr_3-6196899,pr_3-6214201,pr_3-6216744,pr_3-6221003,pr_3-6221890,pr_3-6226545,pr_3-6226775,pr_3-6229429,pr_3-6232240,pr_3-6236254,pr_3-6237095,pr_3-6239069,pr_3-6241143,pr_3-6243721,pr_3-6248860,pr_3-6251774,pr_3-6258322,pr_3-6277513,pr_3-6278202,pr_3-6297296,pr_3-6309412,pr_3-6316576,pr_3-6319548,pr_3-6333204,pr_3-6335126,pr_3-6358877,pr_3-6377968,pr_3-6388111,pr_3-6390038,pr_3-6393394,pr_3-6394066,pr_3-6394726,pr_3-6397627,pr_3-6401138,pr_3-6403885,pr_3-6405007,pr_3-6406494,pr_3-6406958,pr_3-6410169,pr_3-6412766,pr_3-6413337,pr_3-6413586,pr_3-6418387,pr_3-6418563,pr_3-6419588,pr_3-6421311,pr_3-6421744,pr_3-6423881,pr_3-6430089,pr_3-6431931,pr_3-6433852,pr_3-6436952,pr_3-6437302,pr_3-6437365,pr_3-6438298,pr_3-6438394,pr_3-6439138,pr_3-6439728,pr_3-6441394,pr_3-6442814,pr_3-6445607,pr_3-6445624,pr_3-6445879,pr_3-6448721,pr_3-6449431,pr_3-6453745,pr_3-6457221,pr_3-6470229,pr_3-6484876,pr_3-6488963,pr_3-6506615,pr_3-6514559,pr_3-6536747,pr_3-6557007,pr_3-6569439
pr_3-6154197,1.000000,0.972096,0.983343,0.982883,0.965365,0.974462,0.975068,0.960435,0.962805,0.960974,0.960873,0.966837,0.965573,0.975303,0.952636,0.962567,0.942651,0.961656,0.962099,0.962338,0.959487,0.957412,0.915516,0.963179,0.947538,0.917385,0.946314,0.950182,0.938905,0.934038,0.945462,0.933794,0.930257,0.897368,0.940517,0.889727,0.936412,0.919702,0.924274,0.930421,0.922536,0.919174,0.932997,0.925383,0.926670,0.917537,0.929414,0.921807,0.916099,0.919699,0.917982,0.921690,0.894228,0.923796,0.921029,0.929256,0.894242,0.928966,0.920161,0.919883,0.890548,0.925916,0.890348,0.919679,0.920928,0.919294,0.894373,0.920347,0.919866,0.918319,0.919884,0.919285,0.922275,0.896263,0.895315,0.879844,0.912337,0.897514,0.904549,0.898446
pr_3-6163984,0.972096,1.000000,0.970255,0.971159,0.977661,0.975785,0.965098,0.961783,0.962902,0.961991,0.975087,0.979309,0.978349,0.962782,0.953594,0.974475,0.936887,0.973296,0.963789,0.963216,0.962241,0.968655,0.913558,0.964796,0.949351,0.917314,0.952226,0.951102,0.940380,0.936277,0.939642,0.928610,0.938502,0.895809,0.937369,0.888290,0.938102,0.930944,0.931789,0.932192,0.934068,0.926201,0.927991,0.921087,0.935320,0.925370,0.930929,0.931223,0.928130,0.931318,0.929273,0.931300,0.892953,0.926075,0.922340,0.926180,0.892112,0.931560,0.922310,0.921250,0.889973,0.923443,0.890668,0.930935,0.923352,0.930074,0.894616,0.917293,0.930882,0.920057,0.921733,0.920532,0.921174,0.895724,0.894641,0.878585,0.913667,0.900185,0.903612,0.897598
pr_3-6175988,0.983343,0.970255,1.000000,0.980278,0.963383,0.972590,0.972887,0.959129,0.961308,0.959747,0.959565,0.965594,0.964135,0.972826,0.951103,0.960323,0.941252,0.959867,0.960208,0.960339,0.957981,0.956170,0.914543,0.962164,0.946859,0.916919,0.945336,0.949797,0.937739,0.933540,0.944812,0.933151,0.930009,0.896230,0.939666,0.888372,0.935835,0.919569,0.924613,0.929710,0.921282,0.918958,0.932203,0.925405,0.926134,0.917656,0.928898,0.920491,0.916001,0.919043,0.916580,0.920343,0.893571,0.923251,0.920559,0.929055,0.892292,0.928378,0.920037,0.919710,0.889928,0.925216,0.890291,0.918177,0.920777,0.918072,0.893875,0.920347,0.918216,0.918393,0.920041,0.918835,0.922391,0.895732,0.894017,0.879925,0.911539,0.897758,0.904629,0.898339
pr_3-6181787,0.982883,0.971159,0.980278,1.000000,0.965164,0.974679,0.974960,0.961210,0.961817,0.959910,0.959687,0.966585,0.965246,0.975581,0.955057,0.963071,0.959749,0.962106,0.963176,0.963247,0.958844,0.956734,0.934120,0.964756,0.950469,0.935453,0.946138,0.952873,0.940040,0.938271,0.949210,0.939722,0.935453,0.915370,0.944794,0.907365,0.940931,0.920311,0.930575,0.934935,0.923052,0.926468,0.938361,0.930673,0.931477,0.924253,0.934025,0.921865,0.917316,0.919804,0.918085,0.921851,0.911506,0.927599,0.927986,0.935401,0.911974,0.933739,0.927081,0.925822,0.909418,0.931283,0.909047,0.919885,0.927103,0.919408,0.912915,0.925601,0.920386,0.925173,0.926540,0.925976,0.927178,0.914048,0.912723,0.898412,0.917122,0.903474,0.910868,0.903881
pr_3-6195263,0.965365,0.977

In [15]:
#========================================================================
# Get Low Corr Model For Ensemble
select_list = []
thres_corr = 0.97
# thres_corr = 0.985
base_model = top_corr.iloc[0].index[0]
select_list.append(base_model)

def ensemble_corr_checker(model):
    df_corr = top_corr.loc[model].reset_index()
    
    current_idx = list(top_corr.index).index(model)
    df_corr = df_corr.iloc[current_idx:, ]
    best_cv = 100
    best_corr = 0
    for _, rows in df_corr.iterrows():
        cv = rows[0]
        corr = rows[1]
        if cv.count('3-60') or cv.count('3-7') or cv.count('3-8'):
            continue
        
        if cv not in select_list and corr<thres_corr:
            tmp = top_corr.loc[select_list, cv]
            cnt = tmp[tmp>=thres_corr].sum()
            if cnt==0:
                return cv, corr
    
    return '', 0

# cv, corr = ensemble_corr_checker(base_model)
# if corr>=thres_corr:
#     continue
    
is_no_stack = 0
is_loop = True
while is_loop:
    for model in select_list:    
        cv, corr = ensemble_corr_checker(base_model)
        if len(select_list)==1:
            if corr:
                select_list.append(cv)
        elif len(select_list)>1:
            cv, corr = ensemble_corr_checker(select_list[-1])
            if corr:
                select_list.append(cv)
            else:
                is_loop=False
#========================================================================

In [16]:
print(len(select_list))
print(select_list)
df_select = top_corr.loc[select_list, select_list]
# tmp = df_select['lb3664'].iloc[:9]
# select_list = list(tmp.index)
# select_list
df_select

45
['pr_3-6154197', 'pr_3-6195263', 'pr_3-6216744', 'pr_3-6221003', 'pr_3-6236254', 'pr_3-6239069', 'pr_3-6243721', 'pr_3-6277513', 'pr_3-6297296', 'pr_3-6316576', 'pr_3-6319548', 'pr_3-6333204', 'pr_3-6335126', 'pr_3-6358877', 'pr_3-6377968', 'pr_3-6388111', 'pr_3-6390038', 'pr_3-6394066', 'pr_3-6397627', 'pr_3-6401138', 'pr_3-6405007', 'pr_3-6406494', 'pr_3-6410169', 'pr_3-6418387', 'pr_3-6418563', 'pr_3-6419588', 'pr_3-6421311', 'pr_3-6421744', 'pr_3-6423881', 'pr_3-6430089', 'pr_3-6436952', 'pr_3-6438394', 'pr_3-6439728', 'pr_3-6441394', 'pr_3-6445607', 'pr_3-6445624', 'pr_3-6448721', 'pr_3-6470229', 'pr_3-6484876', 'pr_3-6488963', 'pr_3-6506615', 'pr_3-6514559', 'pr_3-6536747', 'pr_3-6557007', 'pr_3-6569439']


,pr_3-6154197,pr_3-6195263,pr_3-6216744,pr_3-6221003,pr_3-6236254,pr_3-6239069,pr_3-6243721,pr_3-6277513,pr_3-6297296,pr_3-6316576,pr_3-6319548,pr_3-6333204,pr_3-6335126,pr_3-6358877,pr_3-6377968,pr_3-6388111,pr_3-6390038,pr_3-6394066,pr_3-6397627,pr_3-6401138,pr_3-6405007,pr_3-6406494,pr_3-6410169,pr_3-6418387,pr_3-6418563,pr_3-6419588,pr_3-6421311,pr_3-6421744,pr_3-6423881,pr_3-6430089,pr_3-6436952,pr_3-6438394,pr_3-6439728,pr_3-6441394,pr_3-6445607,pr_3-6445624,pr_3-6448721,pr_3-6470229,pr_3-6484876,pr_3-6488963,pr_3-6506615,pr_3-6514559,pr_3-6536747,pr_3-6557007,pr_3-6569439
pr_3-6154197,1.000000,0.965365,0.960435,0.962805,0.952636,0.942651,0.962099,0.915516,0.947538,0.946314,0.950182,0.938905,0.934038,0.945462,0.933794,0.930257,0.897368,0.889727,0.919702,0.924274,0.922536,0.919174,0.925383,0.921807,0.916099,0.919699,0.917982,0.921690,0.894228,0.923796,0.894242,0.890548,0.890348,0.919679,0.919294,0.894373,0.919866,0.922275,0.896263,0.895315,0.879844,0.912337,0.897514,0.904549,0.898446
pr_3-6195263,0.965365,1.000000,0.963512,0.965474,0.955198,0.931878,0.965677,0.910793,0.950326,0.953347,0.951347,0.941613,0.936828,0.932180,0.920972,0.939026,0.893161,0.883971,0.925461,0.931947,0.927711,0.926710,0.915074,0.927727,0.922832,0.924802,0.924723,0.927576,0.889774,0.926369,0.888773,0.886321,0.886589,0.925719,0.925267,0.890436,0.925124,0.916843,0.892348,0.890959,0.874715,0.913465,0.901434,0.900061,0.895678
pr_3-6216744,0.960435,0.963512,1.000000,0.963711,0.957339,0.931845,0.966408,0.918132,0.955625,0.951274,0.957567,0.966394,0.941277,0.936206,0.927760,0.941306,0.898918,0.891389,0.914053,0.942985,0.915304,0.931617,0.927890,0.915875,0.911909,0.916351,0.913469,0.914606,0.894532,0.949906,0.896372,0.892954,0.892628,0.915305,0.915794,0.895350,0.914546,0.923721,0.899771,0.898267,0.880084,0.923477,0.912785,0.908745,0.908951
pr_3-6221003,0.962805,0.965474,0.963711,1.000000,0.954998,0.928477,0.965896,0.908045,0.950890,0.957253,0.949723,0.942670,0.935267,0.927277,0.915510,0.930557,0.888687,0.880548,0.911111,0.924204,0.912835,0.917565,0.911726,0.914387,0.910391,0.912540,0.911519,0.912370,0.885379,0.925284,0.885329,0.883613,0.881939,0.911815,0.913031,0.885018,0.912135,0.912493,0.888856,0.887222,0.869527,0.912898,0.898640,0.894792,0.891342
pr_3-6236254,0.952636,0.955198,0.957339,0.954998,1.000000,0.929134,0.960624,0.913862,0.967603,0.941974,0.950743,0.936978,0.966991,0.930185,0.926758,0.935094,0.893106,0.886009,0.903109,0.931472,0.905116,0.928939,0.916274,0.905005,0.904606,0.904534,0.903524,0.905483,0.890375,0.920856,0.889925,0.887623,0.888147,0.905910,0.906263,0.891046,0.905762,0.914643,0.895061,0.891074,0.874590,0.932237,0.905487,0.904225,0.896251
pr_3-6239069,0.942651,0.931878,0.931845,0.928477,0.929134,1.000000,0.935531,0.964915,0.923496,0.918395,0.933115,0.910748,0.917035,0.924311,0.925654,0.917617,0.947242,0.939801,0.884467,0.917910,0.889440,0.917167,0.914694,0.886185,0.884990,0.886460,0.884671,0.888448,0.942317,0.900635,0.943483,0.943259,0.942950,0.886753,0.885460,0.946274,0.888285,0.902903,0.941752,0.944429,0.930143,0.901934,0.890126,0.895847,0.887368
pr_3-6243721,0.962099,0.965677,0.966408,0.965896,0.960624,0.935531,1.000000,0.919394,0.954714,0.956737,0.960260,0.944988,0.947957,0.934906,0.927612,0.944318,0.899588,0.890892,0.911678,0.935897,0.913566,0.933872,0.919043,0.915131,0.912237,0.913385,0.911362,0.914080,0.897622,0.932962,0.895724,0.895481,0.894853,0.914746,0.912517,0.897574,0.913316,0.926304,0.900406,0.897888,0.881313,0.925935,0.912709,0.914357,0.908655
pr_3-6277513,0.915516,0.910793,0.918132,0.908045,0.913862,0.964915,0.919394,1.000000,0.912934,0.903136,0.929121,0.900716,0.909623,0.915166,0.921586,0.916814,0.960457,0.953503,0.870658,0.919592,0.871500,0.921395,0.911599,0.871444,0.868877,0.871379,0.869887,0.873803,0.948941,0.897831,0.955162,0.948733,0.949816,0.872446,0.870653,0.956957,0.873553,0.904338,0.953450,0.952825,0.940247,0.899680,0.891720,0.903878,0.896223
pr_3-6297296,0.947538,0.950326,0.955625,0.950890,0.967603,0.923496,0.9

In [17]:
# ================================================================
# 相関行列から移動するpathを決める
import shutil
for path in ens_list:
    for sel in select_list:
#         sel_key = sel[3:].replace('-', '.')
        sel_key = sel[3:]
        if path.count(sel_key):
#             shutil.move(path, '../ensemble/lgb_ensemble/')
            try:
                shutil.move(path, '../ensemble/dir_stack_blend/tmp')
#                 shutil.move(path, '../ensemble/rm_outlier_ensemble/tmp/')
            except FileNotFoundError:
                pass
            pass
        else:
            pass
#             shutil.move(path, '../ensemble/used_stack/')
# ================================================================

100%|██████████| 1/1 [00:00<00:00,  2.90it/s]


,target,clf_pred,no_out_flg
card_id,,,
C_ID_92a2005557,-0.820283,0.000444,1.0
C_ID_3d0044924f,0.392913,0.007831,0.0
C_ID_d639edf6cd,0.688056,0.004074,0.0
C_ID_186d6a6901,0.142495,0.000797,0.0
C_ID_cdbd2c0db2,-0.159749,0.000251,1.0
